# Django REST Framework
The Django REST Framework (DRF) is a powerful and flexible toolkit for building Web APIs in Django. It provides a set of tools and libraries that make it easy to create RESTful APIs, handle serialization, authentication, and permissions.



## Installation

Django REST Framework is not included in Django by default. You need to install it separately.

Since this project uses Pipenv, install DRF as follows:

In [ ]:
pipenv install djangorestframework

This will add `djangorestframework` to your Pipfile under [packages].

## Setup

After installation, you need to add DRF to your Django project's settings.

1. Open `config/settings.py`.
2. Add `'rest_framework'` to the `INSTALLED_APPS` list.

In [ ]:
# In config/settings.py
INSTALLED_APPS = [
    ...,
    'rest_framework',  # Add this line
]

You can also configure DRF settings in `settings.py`. For example, to enable pagination:

In [ ]:
# In config/settings.py
REST_FRAMEWORK = {
    'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.PageNumberPagination',
    'PAGE_SIZE': 10,
}

## Features

Django REST Framework provides powerful tools for building Web APIs:

- **Serializers**: Convert complex data types (like Django models) to Python data types that can be easily rendered into JSON, XML, etc.
- **ViewSets**: A set of views that provide CRUD operations for a model.
- **Routers**: Automatically generate URL patterns for ViewSets.
- **Authentication**: Support for various authentication methods (Token, JWT, OAuth, etc.).
- **Permissions**: Control access to API endpoints.
- **Pagination**: Limit the number of results returned in a single response.
- **Filtering**: Allow clients to filter results.
- **Browsable API**: A web interface for testing your API.
- **Throttling**: Rate limiting for API requests.
- **Versioning**: Support for API versioning.
- **Schema Generation**: Automatic API schema generation for documentation.

## Serializers

Serializers allow you to convert Django model instances to JSON and vice versa.

Create a `serializers.py` file in your app (e.g., `apps/user_management/serializers.py`):

In [ ]:
# apps/user_management/serializers.py
from rest_framework import serializers
from django.contrib.auth import get_user_model
from django.contrib.auth.models import Group

User = get_user_model()

class UserSerializer(serializers.HyperlinkedModelSerializer):
    class Meta:
        model = User
        fields = ['url', 'username', 'email', 'groups']

class GroupSerializer(serializers.HyperlinkedModelSerializer):
    class Meta:
        model = Group
        fields = ['url', 'name']

This creates serializers for your custom User model and the Group model.

## ViewSets

ViewSets provide a way to define the view logic for your API endpoints.

Update `apps/user_management/views.py`:

In [ ]:
# apps/user_management/views.py
from rest_framework import viewsets, permissions
from django.contrib.auth import get_user_model
from django.contrib.auth.models import Group
# from .serializers import UserSerializer, GroupSerializer

User = get_user_model()

class UserViewSet(viewsets.ModelViewSet):
    """
    API endpoint that allows users to be viewed or edited.
    """
    queryset = User.objects.all().order_by('-date_joined')
    serializer_class = UserSerializer
    permission_classes = [permissions.IsAuthenticated]

class GroupViewSet(viewsets.ModelViewSet):
    """
    API endpoint that allows groups to be viewed or edited.
    """
    queryset = Group.objects.all().order_by('name')
    serializer_class = GroupSerializer
    permission_classes = [permissions.IsAuthenticated]

## URLs

Use routers to automatically generate URL patterns for your ViewSets.

Update `config/urls.py`:

In [ ]:
# config/urls.py
from django.contrib import admin
from django.urls import path, include
from rest_framework import routers
from apps.user_management import views

router = routers.DefaultRouter()
router.register(r'users', views.UserViewSet)
router.register(r'groups', views.GroupViewSet)

urlpatterns = [
    path('admin/', admin.site.urls),
    path('api/', include(router.urls)),
    path('api-auth/', include('rest_framework.urls', namespace='rest_framework')),
]

This sets up API endpoints at `/api/users/` and `/api/groups/`, and includes login/logout views for the browsable API.

## Testing the API

Run the Django development server and test your API.

In [ ]:
# Run the server
python manage.py runserver

Visit `http://127.0.0.1:8000/api/users/` in your browser to see the browsable API.

You can also test with curl or httpie:

```bash
curl -u username:password http://127.0.0.1:8000/api/users/
```

Or with httpie:

```bash
http -a username http://127.0.0.1:8000/api/users/
```

## Additional Features

### Authentication

DRF supports various authentication methods to verify user identity. You can set default authentication globally or per view.

#### Setting Global Authentication

In `config/settings.py`:

```python
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': [
        'rest_framework.authentication.BasicAuthentication',
        'rest_framework.authentication.SessionAuthentication',
    ]
}
```

#### Token Authentication

For token-based auth, install `djangorestframework-simplejwt`:

```bash
pipenv install djangorestframework-simplejwt
```

Add to settings:

```python
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': [
        'rest_framework_simplejwt.authentication.JWTAuthentication',
    ]
}
```

Include token URLs in `config/urls.py`:

```python
from rest_framework_simplejwt.views import (
    TokenObtainPairView,
    TokenRefreshView,
)

urlpatterns = [
    # ... other paths ...
    path('api/token/', TokenObtainPairView.as_view(), name='token_obtain_pair'),
    path('api/token/refresh/', TokenRefreshView.as_view(), name='token_refresh'),
]
```

#### Per-View Authentication

```python
from rest_framework.authentication import SessionAuthentication, BasicAuthentication
from rest_framework.permissions import IsAuthenticated
from rest_framework.response import Response
from rest_framework.views import APIView

class ExampleView(APIView):
    authentication_classes = [SessionAuthentication, BasicAuthentication]
    permission_classes = [IsAuthenticated]

    def get(self, request, format=None):
        content = {
            'user': str(request.user),
            'auth': str(request.auth),
        }
        return Response(content)
```

### Permissions

Permissions control access to API endpoints. They run before views and can be global or per-view.

#### Built-in Permission Classes

- `AllowAny`: Unrestricted access
- `IsAuthenticated`: Only authenticated users
- `IsAdminUser`: Only staff users
- `IsAuthenticatedOrReadOnly`: Authenticated for write, anyone for read

#### Setting Global Permissions

```python
REST_FRAMEWORK = {
    'DEFAULT_PERMISSION_CLASSES': [
        'rest_framework.permissions.IsAuthenticated',
    ]
}
```

#### Per-View Permissions

```python
from rest_framework.permissions import IsAuthenticated
from rest_framework.response import Response
from rest_framework.views import APIView

class ExampleView(APIView):
    permission_classes = [IsAuthenticated]

    def get(self, request, format=None):
        content = {
            'status': 'request was permitted'
        }
        return Response(content)
```

#### Custom Permissions

Create a custom permission class:

```python
from rest_framework import permissions

class IsOwnerOrReadOnly(permissions.BasePermission):
    """
    Custom permission to only allow owners of an object to edit it.
    """

    def has_object_permission(self, request, view, obj):
        # Read permissions are allowed to any request
        if request.method in permissions.SAFE_METHODS:
            return True

        # Write permissions are only allowed to the owner
        return obj.owner == request.user
```

Use it in a ViewSet:

```python
class MyModelViewSet(viewsets.ModelViewSet):
    permission_classes = [permissions.IsAuthenticated, IsOwnerOrReadOnly]
    # ... other attributes
```

### Filtering and Searching

DRF provides powerful filtering capabilities to narrow down API results.

#### Basic Filtering
For simple filtering, you can override `get_queryset()` in your view but it only works for very basic cases.

```python
class UserListView(generics.ListAPIView):
    serializer_class = UserSerializer

    def get_queryset(self):
        queryset = User.objects.all()
        username = self.request.query_params.get('username')
        if username is not None:
            queryset = queryset.filter(username=username)
        return queryset
```

#### DjangoFilterBackend
The `DjangoFilterBackend` provides more advanced filtering capabilities that work well for complex queries. Using it we can easily filter based on model fields.

Install `django-filter`:

```bash
pipenv install django-filter
```

Add to settings:

```python
INSTALLED_APPS = [
    # ... other apps
    'django_filters',
]

REST_FRAMEWORK = {
    'DEFAULT_FILTER_BACKENDS': ['django_filters.rest_framework.DjangoFilterBackend']
}
```

Use in a view:

```python
from django_filters.rest_framework import DjangoFilterBackend

class ProductList(generics.ListAPIView):
    queryset = Product.objects.all()
    serializer_class = ProductSerializer
    filter_backends = [DjangoFilterBackend]
    filterset_fields = ['category', 'in_stock']
```
The above example allows filtering products by `category` and `in_stock` status. For example, to get all products in category "electronics" that are in stock, you would make a request to:

```
/api/products/?category=electronics&in_stock=True
```

#### SearchFilter

Add search functionality:

```python
from rest_framework import filters

class UserListView(generics.ListAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer
    filter_backends = [filters.SearchFilter]
    search_fields = ['username', 'email']
```
The above example allows searching users by `username` or `email` using a query parameter like:

```
/api/users/?search=john
```

#### OrderingFilter

Allow ordering results:

```python
class UserListView(generics.ListAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer
    filter_backends = [filters.OrderingFilter]
    ordering_fields = ['username', 'email']
    ordering = ['username']
```
The above example allows ordering users by `username` or `email`. You can specify the ordering in the query parameter with both ascending(default) and descending order(using -):
```
/api/users/?ordering=-email //descending
/api/users/?ordering=email //ascending
```

### Documentation

Generate automatic API documentation.

#### DRF Spectacular

Install `drf-spectacular`:

```bash
pipenv install drf-spectacular
```

Add to settings:

```python
INSTALLED_APPS = [
    # ... other apps
    'drf_spectacular',
]

REST_FRAMEWORK = {
    'DEFAULT_SCHEMA_CLASS': 'drf_spectacular.openapi.AutoSchema',
}
```

Add to URLs:

```python
from drf_spectacular.views import SpectacularAPIView, SpectacularRedocView, SpectacularSwaggerView

urlpatterns = [
    # ... other paths
    path('api/schema/', SpectacularAPIView.as_view(), name='schema'),
    path('api/schema/swagger-ui/', SpectacularSwaggerView.as_view(url_name='schema'), name='swagger-ui'),
    path('api/schema/redoc/', SpectacularRedocView.as_view(url_name='schema'), name='redoc'),
]
```

### Throttling

Limit the rate of API requests.

#### Setting Throttling

```python
REST_FRAMEWORK = {
    'DEFAULT_THROTTLE_CLASSES': [
        'rest_framework.throttling.AnonRateThrottle',
        'rest_framework.throttling.UserRateThrottle'
    ],
    'DEFAULT_THROTTLE_RATES': {
        'anon': '100/hour',
        'user': '1000/hour'
    }
}
```

### Validation

DRF provides robust validation for incoming data.

#### Serializer Validation

```python
class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = ['username', 'email', 'password']

    def validate_username(self, value):
        if len(value) < 3:
            raise serializers.ValidationError("Username must be at least 3 characters long.")
        return value
```

### Versioning

Handle API versioning.

#### URL Path Versioning

```python
REST_FRAMEWORK = {
    'DEFAULT_VERSIONING_CLASS': 'rest_framework.versioning.URLPathVersioning'
}
```

Add to URLs:

```python
urlpatterns = [
    path('api/<version>/users/', UserListView.as_view()),
]
```